# ["On the predictability of infectious disease outbreaks"](https://www.nature.com/articles/s41467-019-08616-0), Samuel V. Scarpino & Giovanni Petri

In order to study the predictability of diseases (more precisely time-series) in a comparative framework, the authors of the paper employ **Permutation Entropy** as a model-free measure of time-series predictability. It works by first converting the time-series into a symbolic representation. Let us consider a simple example of how it works. Imagine we have a signal (time-series):

$$X = \{120, 74, 203, 167, 92, 148, 174, 47\}$$

We shall transform this series into symbol series. For simplicity, let us suppose that we use an embedding dimension $m = 3$. This quantity determines the amount of symbols that can possibly exist. See in the figure as an illustration of the possible symbols that can be obtained from this time-series.

<img src="./images/PE.png" width="400">

The first step to transform $X$ into symbol sequences is to sort their subchains of length $m$ in increasing order. So, we take the first three elements of $X$ and sort them, which leaves us with $\{74, 120, 203\}$. We have kept tract of these values' indices, such that the sequence now looks like $\{2, 1, 3\}$. This first subchain maps to symbol $D$. From this scheme, we just need to advance one value at a time: the next subchain to consider is $\{74, 203, 167\}$. Its sorted version is $\{74, 167, 203\}$ which corresponds to $\{1, 3, 2\}$, and maps to $B$. And so on, until to achieve a symbol sequence $\hat{X} = \{D, B, F, E, A, C\}$. Let's define the ensemble of all $m!$ possible symbols as $\Pi$. In our case, we consider directly the same permutations as symbols, for example the following permutations are derived by an embedding dimension $m = 3$: $\{(1, 2, 3): 0, (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)\}$.

The permutation entropy (PE) of the time-series $X$ is given by the following Shannon entropy that exploit the symbolic representaion of the time-serie: 

$$H^p(X) = - \sum_\pi p_{\pi} log(p_{\pi})$$ 

where $p_{\pi}$ is the probability of encountering the symbol (or pattern/permutation) $\pi \in \Pi$ in the symbolic representation of the time-series $X$. Furthermore, to control for differences in dimension and for the effect of time-series length on the entropy estimation, we normalize the entropy by the log number of observed symbols (this type of normalization, and not for $m!$, allows to find a minimum for the embedding dimension, see later).

Finally, the metric used is the predictability defined as $\chi = 1 - H^p$. The closer to 1 the $\chi$ is, the more regular and more deterministic the time series is. Contrarily, the smaller $\chi$ is, the more noisy and random the time series is. A time-serie that visits all the possible symblos with equal frequency will have maximal entropy and minimal predictability, and a time-series that only samples a few of the possible symbols will instead have lower entropy and hence be more predictable.

N.B. $\tau$ value is fixed to $1$ (see paper for more detail) in this notebook and this notebook works fine for time-series dataset without NaN values.

In [21]:
import PermutationEntropy
import plotly.graph_objects as go
from tqdm import tqdm
import pandas as pd
import numpy as np
import math

# In this code I employ the parallelization of the processes on 8 cores.
#from dask.diagnostics import ProgressBar
#import dask.dataframe as dd

In [22]:
# Set if you want to parallelizate the process of this notebook.
parallelization = False

In [23]:
# Load the second version of the daily data (interpolate nan values) of the fcs indicator.
fcs = pd.read_csv("../../Data Sources/Food Consumption Score (FCS)/time-series/Yemen/wfp_fcs-v2-daily-interpolate.csv", header = [0, 1], index_col = 0)
fcs.columns.names = ["AdminStrata", "Indicator"]
fcs.index.name = "Datetime"
fcs.index = pd.to_datetime(fcs.index)
freq = "D"
fcs.index.freq = freq
fcs.head()

AdminStrata,Abyan,Aden,Al Bayda,Al Dhale'e,Al Hudaydah,Al Jawf,Al Maharah,Al Mahwit,Amanat Al Asimah,Amran,...,Hadramaut,Hajjah,Ibb,Lahj,Marib,Raymah,Sa'ada,Sana'a,Shabwah,Taizz
Indicator,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,...,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS
Datetime,,,,,,,,,,,,,,,,,,,,,
2018-07-02,31.158690,16.619519,38.332669,29.194825,20.788151,22.085706,2.892308,16.815145,17.000398,20.446735,...,23.822825,28.361345,25.036668,31.762436,32.596233,54.121774,17.676022,26.917713,16.752289,26.563365
2018-07-03,32.675222,16.370603,43.292084,31.336940,20.692544,23.477196,2.913825,19.833443,17.208166,21.689014,...,23.699881,28.805448,27.102390,32.049499,33.333333,56.722689,17.134560,25.783476,18.257453,29.124005
2018-07-04,33.415597,15.089163,45.128205,33.135157,21.825051,24.864000,2.950408,19.614289,18.070801,25.611124,...,23.103187,30.993706,29.844066,33.106267,36.883683,56.306306,17.454545,25.436047,19.083081,29.427973
2018-07-05,33.531451,15.766521,43.959297,34.554309,21.481693,27.814992,2.247913,19.558566,19.021964,27.922484,...,21.596419,32.905902,29.025363,32.938828,36.897633,56.231003,17.846350,27.421759,20.349533,30.456026
2018-07-06,33.951856,16.870065,44.516521,33.878557,21.076712,28.688245,2.269780,17.608130,19.817567,28.712235,...,20.129084,32.598181,30.692766,33.837934,36.669568,57.324841,18.463057,26.547231,21.107524,31.012517


What value of the embedding dimension use for the these time-series? In order to find the appropriate embedding dimension for clustering a set of time-series, we calculate the average normalized entropy of a set of distributions. The value of the $H^p$ should always decline as the embedding dimension grows, i.e. no minimum value of $H^p$ will exist for finite time-series. To address this issue, we follow Brandmaier and exclude all unobserved symbols when calculating $H^p$, which acts as a penalty against higher dimensions and results in a minimum value of $H^p$ for finite length time-series. To control for differences in dimension and for the effect of time-series length on the entropy estimation, we normalize the entropy by the log number of observed symbols.

The embedding dimension $m$ of a time-series is the length of the basic blocks used in the calculation of the permutation entropy, *it encodes the fundamental temporal unit of predictability* in the form of an entropy production rate. The result that predictability depends on temporal scale also suggests that the permutation entropy could be an approach for justifying the utility of different data sets, i.e. **one could determine the optimal granularity of data by selecting the dimension that maximized predictability**.

In [24]:
ms = np.arange(2, 21)

In [34]:
df = fcs.copy()

In [30]:
if parallelization:
    # The Dask library doesn't support pandas apply on axis = 0 -> transpose the dataframe to do the same thing on axis = 0.
    ddata = dd.from_pandas(df.transpose(), npartitions = 8)

    with ProgressBar():
        %time embedding_search = ddata.map_partitions(lambda df: df.apply((lambda row: PermutationEntropy.search_best_m(row, ms)), axis = 1)).compute(scheduler = "processes")
        embedding_search = embedding_search.transpose()
else:
    %time embedding_search = df.apply(lambda column: PermutationEntropy.search_best_m(column, ms))

Wall time: 15.2 s


In [31]:
embedding_search.idxmin()

AdminStrata       Indicator
Abyan             rCSI         4
Aden              rCSI         4
Al Bayda          rCSI         4
Al Dhale'e        rCSI         4
Al Hudaydah       rCSI         4
Al Jawf           rCSI         4
Al Maharah        rCSI         4
Al Mahwit         rCSI         4
Amanat Al Asimah  rCSI         4
Amran             rCSI         3
Dhamar            rCSI         4
Hadramaut         rCSI         4
Hajjah            rCSI         4
Ibb               rCSI         4
Lahj              rCSI         4
Marib             rCSI         4
Raymah            rCSI         4
Sa'ada            rCSI         4
Sana'a            rCSI         4
Shabwah           rCSI         4
Taizz             rCSI         4
dtype: int64

In [21]:
#best_m = embedding_search.sum(axis = 1).idxmin()
#best_m

Now, we turn our attention to the FCS indicator (composed by adminstrata-level time-series) and ask how the predictability, defined as $\chi = 1 - H^p$, scales with the amount of available data (i.e. the time-series length). Specifically, we compute the permutation entropy across the FCS dataset of the Yemen country and plotting the predictability as a function of the length of each time-series. Focusing on the predictability over short timescales for each time-serie, we average $H^p$ over temporal windows of width up to 100 days by selecting 1000 random points from each adminstrata-level time-series and calculating $H^p$ for windows of length 10, 12, ..., 100 days. Plotting, the solid lines indicate the mean value and the shaded region marks the interquartile range across all adminstrata and starting locations in the time-series.

In the paper, they find that all diseases show a clear decrease in predictability with increasing time-series length, which implies that accumulating longer stretches of time-series data for a given disease does not translate into improved predictability.

In [25]:
df = fcs.copy()

In [26]:
# Define the number of iteration you want to perform.
n_iter = 2000
# The min length of the temporal window to generate the subsample time-series.
min_window_length = 10
# The max length of the temporal window to generate the subsample time-series.
max_window_length = 100

ms = np.arange(2, 7 + 1) 

In [27]:
if parallelization:
    ddata = dd.from_pandas(df.transpose(), npartitions = 8)
    with ProgressBar():
        %time results = ddata.map_partitions(lambda df: df.apply((lambda row: PermutationEntropy.PE_scaling_with_amount_of_data(row, n_iter, ms, min_window_length, max_window_length)), axis = 1)).compute(scheduler = "processes")
else:
    tqdm.pandas()
    %time results = df.progress_apply(lambda column: PermutationEntropy.PE_scaling_with_amount_of_data(column, n_iter, ms, min_window_length, max_window_length)).transpose()

100%|███████████████████████████████████████████████████████████████████████████████| 21/21 [2:05:59<00:00, 359.97s/it]

Wall time: 2h 5min 59s


In [28]:
results = pd.concat(np.array_split(results, n_iter, axis = 1), axis = 0)
results.head()

,,PE,n,m
AdminStrata,Indicator,,,
Abyan,FCS,0.927309,73.0,4.0
Aden,FCS,0.885883,93.0,4.0
Al Bayda,FCS,0.919467,72.0,4.0
Al Dhale'e,FCS,0.820801,29.0,4.0
Al Hudaydah,FCS,0.849398,97.0,4.0


In [29]:
y = results["PE"]
x = results["n"]

ent = y.groupby(by = x).apply(np.mean).values
ent_low = y.groupby(by = x).apply(lambda x: x.quantile(q = 0.25)).values
ent_high = y.groupby(by = x).apply(lambda x: x.quantile(q = 0.75)).values
n = y.groupby(by = x).apply(np.mean).index

results = pd.DataFrame({"Chi": 1 - ent, "n": n, "Chimax": 1 - ent_low, "Chimin": 1 - ent_high})

In [30]:
results.head()

,Chi,n,Chimax,Chimin
0,0.210657,10.0,0.262798,0.081704
1,0.199959,11.0,0.278072,0.118709
2,0.207601,12.0,0.314903,0.101756
3,0.188572,13.0,0.277501,0.081704
4,0.188173,14.0,0.242098,0.088811


In [31]:
# Create figure.
fig = go.Figure()

fig.add_trace(go.Scatter(x = results["n"], y = results["Chimin"], name = "quantile 25", fill = None, mode = "lines", line = dict(width = .5, color = "#B6B6B6")))
fig.add_trace(go.Scatter(x = results["n"], y = results["Chimax"], name = "quantile 75", fill = "tonexty", mode = "lines", line = dict(width = .5, color = "#B6B6B6")))

fig.add_trace(go.Scatter(x = results["n"], y = results["Chi"], name = "mean", mode = "lines", line = dict(width = 1.5, color = "#FF8F17")))

# Edit the layout.
fig.update_layout(title = dict(text = "FCS - predictability", y = 0.9, x = 0.5), 
                  yaxis_title = dict(text = "Predictability (1 - H)"))

# Add range slider.
fig.update_layout(xaxis = dict(title = "n° of days"))

fig.show()

In [32]:
fig.write_image("figure.png", width = 900, height = 550, scale = 2)